**XGBoost solution attempt :)**

In [634]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [635]:
df_train = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv').set_index('id')
df_train

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0


**Counting Mutual Information scores for features**

In [636]:
from sklearn.feature_selection import mutual_info_regression

df_train1 = df_train.fillna(df_train.median())

X1 = df_train1.copy()
y1 = X1.pop("failure")


# Label encoding for categoricals
for colname in X1.select_dtypes("object"):
    X1[colname], _ = X1[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X1.dtypes == int

def make_mi_scores(X1, y1, discrete_features):
    mi_scores = mutual_info_regression(X1, y1, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X1.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X1, y1, discrete_features)
mi_scores  # show a few features with their MI scores

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


measurement_2     0.004359
attribute_2       0.004179
loading           0.003707
attribute_0       0.003265
measurement_17    0.002858
attribute_3       0.002593
measurement_3     0.001896
product_code      0.001763
measurement_12    0.001458
measurement_10    0.001276
measurement_11    0.001022
measurement_7     0.000421
measurement_6     0.000252
measurement_1     0.000000
measurement_0     0.000000
measurement_4     0.000000
measurement_5     0.000000
attribute_1       0.000000
measurement_8     0.000000
measurement_9     0.000000
measurement_13    0.000000
measurement_14    0.000000
measurement_15    0.000000
measurement_16    0.000000
Name: MI Scores, dtype: float64

In [637]:
X = df_train.copy()
y = X.pop("failure")

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

**Preprocessing**

In [638]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer1',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False)),
])


# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

**GridSearchCV parameters grid for XGBoost model**

In [639]:

param_grid = {
#     "classifier__n_estimators": [10, 50, 100],
#     "classifier__learning_rate": [0.001, 0.005, 0.01, 0.05, 0.1],
    "classifier__iterations": [101],
    "classifier__learning_rate": [0.0055],
}


pipe = Pipeline(
#     steps=[("preprocessor", preprocessor), ("classifier", XGBClassifier(n_jobs=4))]
    steps=[("preprocessor", preprocessor), ("classifier", CatBoostClassifier(verbose=False))]
)

grid_search = GridSearchCV(pipe, param_grid=param_grid, n_jobs=4)




**Training model on train dataset**

In [640]:
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer1',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler(with_mean=False))]),
                                                                         ['loading',
                                                                          'attribute_2',
                                                                          'attribute_3',
                                                                          'measurement_0',
                                                         

**Best model params**

In [641]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer1',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  ['loading', 'attribute_2',
                                                   'attribute_3',
                                                   'measurement_0',
                                                   'measurement_1',
                                                   'measurement_2',
                                                   'measurement_3',
                                                   'measurement_4',
                                                   'measurement_5'

**RocAuc score**

In [642]:
from sklearn.metrics import roc_auc_score
model = grid_search.best_estimator_
prediction = model.predict_proba(X_valid)
les_roc = roc_auc_score(y_valid, prediction[:, 1])
les_roc

0.5863576023513801

In [643]:
prediction

array([[0.63346486, 0.36653514],
       [0.64135173, 0.35864827],
       [0.61102446, 0.38897554],
       ...,
       [0.63852697, 0.36147303],
       [0.63625713, 0.36374287],
       [0.62231365, 0.37768635]])

**Training model on full dataset**

In [644]:
grid_search.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer1',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler(with_mean=False))]),
                                                                         ['loading',
                                                                          'attribute_2',
                                                                          'attribute_3',
                                                                          'measurement_0',
                                                         

In [645]:
model = grid_search.best_estimator_

In [646]:
df_test = pd.read_csv('../input/tabular-playground-series-aug-2022/test.csv').set_index('id')
df_test

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
26570,F,119.57,material_5,material_6,6,4,6,9,6,19.305,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
26571,F,113.51,material_5,material_6,6,4,11,8,0,17.883,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
26572,F,112.16,material_5,material_6,6,4,8,12,4,18.475,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
26573,F,112.72,material_5,material_6,6,4,8,11,10,16.518,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
26574,F,208.00,material_5,material_6,6,4,14,16,8,17.808,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,I,144.74,material_7,material_5,9,5,0,4,9,18.465,...,18.573,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466
47341,I,74.53,material_7,material_5,9,5,4,8,7,18.900,...,17.917,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249
47342,I,67.73,material_7,material_5,9,5,10,11,2,18.656,...,19.630,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349


In [647]:
prob = model.predict_proba(df_test)
prob

array([[0.63906997, 0.36093003],
       [0.64388375, 0.35611625],
       [0.64712819, 0.35287181],
       ...,
       [0.65313155, 0.34686845],
       [0.63451212, 0.36548788],
       [0.65230525, 0.34769475]])

In [648]:
df_subb = pd.read_csv('../input/tabular-playground-series-aug-2022/sample_submission.csv')
df_subb

,id,failure
0,26570,0.0
1,26571,0.0
2,26572,0.0
3,26573,0.0
4,26574,0.0
...,...,...
20770,47340,0.0
20771,47341,0.0
20772,47342,0.0
20773,47343,0.0


In [649]:
df_subb.failure = prob[:, 1:2]
df_subb

,id,failure
0,26570,0.360930
1,26571,0.356116
2,26572,0.352872
3,26573,0.352604
4,26574,0.395935
...,...,...
20770,47340,0.372879
20771,47341,0.345441
20772,47342,0.346868
20773,47343,0.365488


**Saving results**

In [650]:
df_subb.to_csv('submission.csv', index = None)